In [1]:
#!/usr/bin/env python --> jupyter
# coding: utf-8
# written by Sheng Lun Kao

# In[1]:
import numpy
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf #SoundFile
import sounddevice as sd
import scipy.signal
from scipy import signal #scipy.signal.firwin_FIR filter
from scipy import optimize #golden search
import matplotlib.pylab as plt
import time #time.sleep
import math
import cmath

In [2]:
#recursive averaging(Step123) RA
def Step123(self, timesignal, fs, i):
    framelength=int(fs*32*10**-3) #fs*32ms
    RA_nfft=framelength*2
    RA_axis = np.arange(int(RA_nfft / 2)) * (fs/RA_nfft)
    overlap=0.5 #50%
    frameNumber=((len(timesignal)-framelength)/(framelength*overlap))+1
    frameNumber=int(frameNumber)
    
    ss=[]
    for j in range(frameNumber):
        if j==0:
            sta=int(j*framelength)
            end=int((j+1)*framelength)
        if j>0:
            sta=int(sta+framelength*overlap)
            end=int(end+framelength*overlap)
        fftx=np.fft.fft(timesignal[sta:end]*np.hanning(framelength), RA_nfft) / int(RA_nfft /8) #RA_nfft=framelength*2
        if j==0:
            ss=abs(fftx)*abs(fftx) #|fftx|*|fftx|
        if j>0:
            newss=0.9*ss+0.1*abs(fftx)*abs(fftx) #s(l)=0.9s(l-1)+0.1s(l)
            ss=newss
        #print("ss=",ss)
    return RA_axis, RA_nfft, ss

#signal to noise ratio(SNR), the larger the better.
def SNR(RA_power, RA_spectrum, f3, fs, RA_nfft):
    signal_e_1_l3_l=RA_spectrum[int(round(f3 / (fs/RA_nfft)))]
    e_1_noise_LR_points = []
    abar_e_1=0
    std_noi_e_1=0
    for aa in range(4):
        e_1_noise_LR_points.append(RA_power[int(round((f3-3*(fs/RA_nfft)-aa*(fs/RA_nfft)) / (fs/RA_nfft)))])
        e_1_noise_LR_points.append(RA_power[int(round((f3+3*(fs/RA_nfft)+aa*(fs/RA_nfft)) / (fs/RA_nfft)))])
    for bb in range(len(e_1_noise_LR_points)):
        abar_e_1=abar_e_1+e_1_noise_LR_points[bb]/(len(e_1_noise_LR_points)/2)
    #print("e_1_noise_LR_points=",e_1_noise_LR_points)
    std_noi_e_1=np.std(e_1_noise_LR_points)
    #noise2sd_e_1=10 * np.log10(np.abs(abar_e_1+2*std_noi_e_1)) + 94
    noise2sd_e_1=10 * np.log10(np.abs(abar_e_1)) + 94
    SNR_e_1=signal_e_1_l3_l-noise2sd_e_1
    return signal_e_1_l3_l, abar_e_1, std_noi_e_1, noise2sd_e_1, SNR_e_1

#Power Spectral Density(PSD) v2
def PSDv2(self, Cathy, AHGLAB, L1, L2, fs, i):
    num_PSDgain_negative=0
    L1L2=L1+L2
    framelength=int(fs*32*10**-3) #fs*32ms
    denoised_Cathy_time = np.zeros(len(Cathy)+framelength)
    RA_nfft=framelength*2
    RA_axis = np.arange(int(RA_nfft / 2)) * (fs/RA_nfft)
    overlap=0.5 #50%
    frameNumber=((len(Cathy)-framelength)/(framelength*overlap))+1
    frameNumber=int(frameNumber)
    print("\nPSDv2 frameNumber=",frameNumber)
    
    ss=[]
    for j in range(frameNumber):
        #print("j=",j)
        if j==0:
            sta=int(j*framelength)
            end=int((j+1)*framelength)
        if j>0:
            sta=int(sta+framelength*overlap)
            end=int(end+framelength*overlap)
        fftx_Cathy=np.fft.fft(Cathy[sta:end]*np.hanning(framelength), RA_nfft) / int(RA_nfft /8) #RA_nfft=framelength*2
        fftx_AHGLAB=np.fft.fft(AHGLAB[sta:end]*np.hanning(framelength), RA_nfft) / int(RA_nfft /8)
        fftx_L1L2=np.fft.fft(L1L2[sta:end]*np.hanning(framelength), RA_nfft) / int(RA_nfft /8)

        if j==0:
            ss_Cathy=abs(fftx_Cathy)*abs(fftx_Cathy) #|fftx|*|fftx|
            ss_AHGLAB=abs(fftx_AHGLAB)*abs(fftx_AHGLAB)
            ss_L1L2=abs(fftx_L1L2)*abs(fftx_L1L2)
        if j>0:
            newss_Cathy=0.9*ss_Cathy+0.1*abs(fftx_Cathy)*abs(fftx_Cathy) #s(l)=0.9s(l-1)+0.1s(l)
            newss_AHGLAB=0.9*ss_AHGLAB+0.1*abs(fftx_AHGLAB)*abs(fftx_AHGLAB)
            newss_L1L2=0.9*ss_L1L2+0.1*abs(fftx_L1L2)*abs(fftx_L1L2)
            ss_Cathy=newss_Cathy
            ss_AHGLAB=newss_AHGLAB
            ss_L1L2=newss_L1L2
    G=1-((ss_AHGLAB+ss_L1L2)/ss_Cathy)
    for qq in range(len(G)):
        if G[qq]<0:
            G[qq]=0.001 
            num_PSDgain_negative=num_PSDgain_negative+1
    RA_PSD_power=ss_Cathy*G
    print("len(G)=",len(G))
    
    denoised_Cathy_time=y=np.fft.ifft(RA_PSD_power, RA_nfft)
    return RA_axis, RA_nfft, RA_PSD_power, denoised_Cathy_time, num_PSDgain_negative

In [3]:
title_font = {'size' : '26'} #標題的字大小
axis_font = {'size' : '20'} #XY軸的字大小
tick_font= {'size' : '14'} #XY刻度的字大小

#name = 'Albert_ER10C_Cathy test-in__ER10C_CMU ref-out__ER10C_AHGLAB ref-in_20201205-1946' #quiet 34 dBA
#name = 'Albert_ER10C_Cathy test-in__ER10C_CMU ref-out__ER10C_AHGLAB ref-in_20201205-2003' #with 2speaker_L&R white noise 70 dBA
name = 'Albert_ER10C_Cathy test-in__ER10C_CMU ref-out__ER10C_AHGLAB ref-in_20201205-2014' #with 2speaker_L&R white noise 75 dBA

savename=name
print("savename= ",savename)

savename=  Albert_ER10C_Cathy test-in__ER10C_CMU ref-out__ER10C_AHGLAB ref-in_20201205-2014


In [4]:
freq_ratio = 1.22
f1 = [600, 800, 900, 1200, 2500, 3200, 3600, 4000, 4800, 6000] #ten pair DPOAE
f2 = [int(i * freq_ratio) for i in f1]
f3 = [int(2 * f1[i] - f2[i]) for i in range(len(f1))]

print("%d pair DPOAE" %len(f1))
print("f1=",f1)
print("f2=",f2)
print("f3=",f3)

10 pair DPOAE
f1= [600, 800, 900, 1200, 2500, 3200, 3600, 4000, 4800, 6000]
f2= [732, 976, 1098, 1464, 3050, 3904, 4392, 4880, 5856, 7320]
f3= [468, 624, 702, 936, 1950, 2496, 2808, 3120, 3744, 4680]


In [5]:
interval = [10, 10, 10, 8, 3, 3, 2, 2, 2, 2]
latency = 0.1

Cathay_amp2pa = 48.81110086394624 #ER10C Cathay 20201205
CMU_amp2pa = 154.62211980677176 #ER10C CMU 20201205
AHGLAB_amp2pa = 36.96203340311423 #ER10C AHGLAB 20201205

In [6]:
%matplotlib qt
my_dpi=96
total_Cathy_signal_l = []
total_CMU_signal_l = []
total_AHGLAB_signal_l = []
total_signal_L1 = []
total_signal_L2 = []

##Computer input pure tone L1_65dB L2_55dB
#L1coef = [1.5033643880137298e-07, 1.6366725153372244e-07, 1.725963049062674e-07, 2.1261368565524393e-07, 1.357374805657057e-07, 1.6366616950089408e-07, 2.1260122636992564e-07, 1.636626522745312e-07, 1.3535048251839166e-07, 2.1261168416772045e-07]
#L2coef = [5.0127877855656796e-08, 5.712135476050241e-08, 6.207146665508096e-08, 5.573571904524015e-08, 4.843786611664115e-08, 5.4454447084449244e-08, 4.490612728584626e-08, 4.287985078190208e-08, 6.02320827596776e-08, 4.297804067343508e-08]

##20201205 Cathy mic recorded L1≌65dB L2≌55dB_quiet 34 dBA
#L1coef =  [1.3865752932333504e-07, 1.4963697561730276e-07, 1.5741550359743908e-07, 1.962997330431317e-07, 1.1756307470982178e-07, 1.332243338827061e-07, 1.6476799799906895e-07, 1.20468242510813e-07, 9.84579799817826e-08, 1.3462647538142852e-07]
#L2coef =  [4.4800709210325965e-08, 5.009662545025079e-08, 5.4606011113025336e-08, 4.897131331417553e-08, 3.926952864086951e-08, 4.118249630196962e-08, 3.512781685344428e-08, 3.118525903144975e-08, 3.906556932547179e-08, 5.5240621549288975e-08]
##Cathy mic recorded L1≌65dB L2≌55dB_white noise 70 dBA
#L1coef =  [1.2958026293714272e-07, 1.3799528555199868e-07, 1.4488311644845396e-07, 1.805378161974351e-07, 1.0602553103539473e-07, 1.1704902740546646e-07, 1.455136279959018e-07, 1.0415470563233654e-07, 8.118706813736941e-08, 1.0686661915946158e-07]
#L2coef =  [4.1569075388032577e-08, 4.6204955342597065e-08, 5.0166375471648715e-08, 4.514664854811448e-08, 3.48871620405534e-08, 3.5571940782784185e-08, 3.0754665958242645e-08, 2.598391044660942e-08, 3.0138753215016455e-08, 8.403434058546602e-08]
##Cathy mic recorded L1≌65dB L2≌55dB_white noise 75 dBA
L1coef =  [1.3712906897447548e-07, 1.4778281966076527e-07, 1.5511120141885243e-07, 1.929324398164241e-07, 1.1384170913035403e-07, 1.2813999130223785e-07, 1.5703645104316956e-07, 1.1343933143296467e-07, 9.219567950268008e-08, 1.2428085214028747e-07]
L2coef =  [4.436217934992272e-08, 4.938246082704739e-08, 5.393507966884611e-08, 4.829245951487835e-08, 3.811419740338272e-08, 3.881888906307747e-08, 3.367530645683877e-08, 2.9377108087614054e-08, 3.62550380115499e-08, 6.281212726104669e-08]

scale_v1 = np.ones(len(f1)) * L1coef
scale_v2 = np.ones(len(f2)) * L2coef
v1 = (2 ** 18) * scale_v1
v2 = (2 ** 18) * scale_v2

for i in range(len(f1)):
    file_l = '/DPOAE_Data/{}_DPOAE_data/{}_left_f{}.wav'.format(name, name, str(i + 1))
    signal_l, fs = sf.read(file_l) #Mic[ER10C_Cathy test-in, ER10C_CMU ref-out, ER10C_AHGLAB ref-in]
    
    Cathy_signal_l = signal_l[:, 0] #ER10C_Cathy test-in
    CMU_signal_l = signal_l[:, 1] #ER10C_CMU ref-out
    AHGLAB_signal_l = signal_l[:, 2] #ER10C_AHGLAB ref-in
    signal_L1 = v1[i] * (np.sin(2*np.pi*np.arange(fs*interval[i])*f1[i]/fs)).astype(np.float32) #Computer input pure tone L1_65dB
    signal_L2 = v2[i] * (np.sin(2*np.pi*np.arange(fs*interval[i])*f2[i]/fs)).astype(np.float32) #Computer input pure tone L2_55dB

    total_Cathy_signal_l.append(Cathy_signal_l[int(latency * fs) : int((latency + interval[i]) * fs)] * Cathay_amp2pa)
    total_CMU_signal_l.append(CMU_signal_l[int(latency * fs) : int((latency + interval[i]) * fs)] * CMU_amp2pa)
    total_AHGLAB_signal_l.append(AHGLAB_signal_l[int(latency * fs) : int((latency + interval[i]) * fs)] * AHGLAB_amp2pa)
    total_signal_L1.append(signal_L1[20:])
    total_signal_L2.append(signal_L2[20:])

In [7]:
print("type(total_Cathy_signal_l) =",type(total_Cathy_signal_l))
print("type(total_Cathy_signal_l[0]) =",type(total_Cathy_signal_l[0]))
print("len(total_Cathy_signal_l) =",len(total_Cathy_signal_l))
print("len(total_Cathy_signal_l[0]) =",len(total_Cathy_signal_l[0]))
print("len(total_signal_L1) =",len(total_signal_L1))
print("len(total_signal_L1[0]) =",len(total_signal_L1[0]))

type(total_Cathy_signal_l) = <class 'list'>
type(total_Cathy_signal_l[0]) = <class 'numpy.ndarray'>
len(total_Cathy_signal_l) = 10
len(total_Cathy_signal_l[0]) = 479980
len(total_signal_L1) = 10
len(total_signal_L1[0]) = 479980


In [8]:
#get quiet_l1l2l3 & RA_quiet_l3_l
origin_l1_l = []
origin_l2_l = []
origin_l3_l = []
RA_origin_l1_l = []
RA_origin_l2_l = []
RA_origin_l3_l = []

for i in range(len(f1)):
    NFFT = len(total_Cathy_signal_l[i])
    df = fs / NFFT
    f_axis = np.arange(int(NFFT / 2)) * df
    
    Cathy_signal_l = total_Cathy_signal_l[i]
    CMU_signal_l = total_CMU_signal_l[i]
    AHGLAB_signal_l = total_AHGLAB_signal_l[i]

    Cathy_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(Cathy_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    
    #Cathy after recursive averaging(Step123), plot spectrum
    RA_axis, RA_nfft, RA_Cathy_signal_l = Step123(0, Cathy_signal_l.real, fs, i)
    RA_Cathy_signal_l_spectrum_l = 10 * np.log10(np.abs(RA_Cathy_signal_l))[:int(RA_nfft / 2)] + 94

    #L1 L2 L3 collection
    origin_l1_l.append(Cathy_spectrum_l[int(round(f1[i] / df))])
    origin_l2_l.append(Cathy_spectrum_l[int(round(f2[i] / df))])
    origin_l3_l.append(Cathy_spectrum_l[int(round(f3[i] / df))])       #quiet_l3_l
    RA_origin_l1_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))])
    RA_origin_l2_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))])
    RA_origin_l3_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))])       #RA_quiet_l3_l

print("\norigin_l1_l = ", origin_l1_l)
print("origin_l2_l = ", origin_l2_l)
print("origin_l3_l = ", origin_l3_l)
print("\nRA_origin_l1_l = ", RA_origin_l1_l)
print("RA_origin_l2_l = ", RA_origin_l2_l)
print("RA_origin_l3_l = ", RA_origin_l3_l)


origin_l1_l =  [64.20130538070534, 64.11324318346627, 64.07223335959705, 64.15628080747581, 63.4720324665459, 62.874515440408615, 62.368694129037884, 61.81628146647682, 61.66501522325395, 60.336341893557055]
origin_l2_l =  [53.938671079419464, 53.735484574924186, 53.7795863469304, 53.75491432154547, 52.918035529973274, 52.06019499392317, 52.50011907228807, 51.71511551574433, 50.5908096787942, 58.29593778748854]
origin_l3_l =  [7.531829667102997, 9.625219156093692, 12.109235235726857, 8.682084200433806, -2.7302628234296265, 4.847100876692139, 7.562633736234119, 11.401369532461516, 4.473925279228752, 11.246725757034866]

RA_origin_l1_l =  [64.87934555892112, 65.70116396996063, 65.96245495384291, 68.00213944980615, 63.46546094323857, 64.45628963573333, 66.04415446165163, 63.45711798155672, 61.61334843574063, 64.24487871738131]
RA_origin_l2_l =  [55.232515976103315, 55.965503948539705, 56.90676542205528, 55.90679200442173, 53.97221050923748, 54.05798193608805, 52.886189534478895, 51.59974

In [9]:
##Power Spectral Density(PSD)

my_dpi=96
numtaps=600 #Length of the filter, FIR filter design using the window method.
SNR_improvement_PSD=[]
beforedenoise_total_SNR_Cathy =[]
beforedenoise_total_noise2sd_Cathy =[]
optimized_total_SNR_PSD = []
optimized_total_noise2sd_PSD = []
L1 = []
L2 = []
origin_l1_l = []
origin_l2_l = []
origin_l3_l = []
RA_origin_l1_l = []
RA_origin_l2_l = []
RA_origin_l3_l = []
RA_PSD_Cathy_l1_l = []
RA_PSD_Cathy_l2_l = []
RA_PSD_Cathy_l3_l = []
totla_num_PSDgain_negative = []

for i in range(len(f1)):
    print("\n--------------  f1=%s  --------------"%(f1[i]))
    NFFT = len(total_Cathy_signal_l[i])
    df = fs / NFFT
    f_axis = np.arange(int(NFFT / 2)) * df
    
    Cathy_signal_l = total_Cathy_signal_l[i]
    CMU_signal_l = total_CMU_signal_l[i]
    AHGLAB_signal_l = total_AHGLAB_signal_l[i]
    signal_L1 = total_signal_L1[i]
    signal_L2 = total_signal_L2[i]
    
    Cathy_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(Cathy_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    CMU_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(CMU_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    AHGLAB_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(AHGLAB_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    spectrum_L1 = 20 * np.log10(np.abs(np.fft.fft(signal_L1, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    spectrum_L2 = 20 * np.log10(np.abs(np.fft.fft(signal_L2, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    #print("\ntype(Cathy_spectrum_l)=",type(Cathy_spectrum_l)) #<class 'numpy.ndarray'>
    print("len(Cathy_spectrum_l)=",len(Cathy_spectrum_l)) #10sec 239990, 2sec 47990

    FIR_bandpassfilter_fdp = signal.firwin(numtaps, [(f3[i]-100)/(fs/2), (f3[i]+100)/(fs/2)], window='hann', pass_zero=False) #FIR_bandpassfilter_fdp, window='hann'
    FIR_Cathy_signal_l = np.convolve(Cathy_signal_l, FIR_bandpassfilter_fdp, mode='same') #convolution
    FIR_CMU_signal_l = np.convolve(CMU_signal_l, FIR_bandpassfilter_fdp, mode='same') #convolution
    FIR_AHGLAB_signal_l = np.convolve(AHGLAB_signal_l, FIR_bandpassfilter_fdp, mode='same') #convolution
    FIR_signal_L1 = np.convolve(signal_L1, FIR_bandpassfilter_fdp, mode='same') #convolution
    FIR_signal_L2 = np.convolve(signal_L2, FIR_bandpassfilter_fdp, mode='same') #convolution 
    
    FIR_Cathy_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(FIR_Cathy_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    FIR_CMU_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(FIR_CMU_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    FIR_AHGLAB_spectrum_l = 20 * np.log10(np.abs(np.fft.fft(FIR_AHGLAB_signal_l, NFFT) / int(NFFT / 2)))[:int(NFFT / 2)] + 94
    
    ##Cathy plot Spectrum & Spectrum (after FIR_bandpassfilter_fdp)
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, Cathy_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], Cathy_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], Cathy_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], Cathy_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_Cathy f1 = {} (Left ear)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], Cathy_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], Cathy_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], Cathy_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], Cathy_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], Cathy_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], Cathy_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_Cathy f1 = %s (Left ear)_%s.png' %(f1[i],savename))

    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, FIR_Cathy_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], FIR_Cathy_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], FIR_Cathy_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], FIR_Cathy_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_Cathy f1 = {} after FIR_bandpassfilter_fdp (Left ear)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], FIR_Cathy_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], FIR_Cathy_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], FIR_Cathy_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], FIR_Cathy_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], FIR_Cathy_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], FIR_Cathy_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_Cathy f1 = %s after FIR_bandpassfilter_fdp (Left ear)_%s.png' %(f1[i],savename))
    
    ##CMU plot Spectrum & Spectrum (after FIR_bandpassfilter_fdp)
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, CMU_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], CMU_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], CMU_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], CMU_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_CMU f1 = {} (ref-out)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], CMU_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], CMU_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], CMU_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], CMU_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], CMU_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], CMU_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_CMU f1 = %s (ref-out)_%s.png' %(f1[i],savename))

    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, FIR_CMU_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], FIR_CMU_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], FIR_CMU_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], FIR_CMU_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_CMU f1 = {} after FIR_bandpassfilter_fdp (ref-out)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], FIR_CMU_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], FIR_CMU_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], FIR_CMU_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], FIR_CMU_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], FIR_CMU_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], FIR_CMU_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_CMU f1 = %s after FIR_bandpassfilter_fdp (ref-out)_%s.png' %(f1[i],savename))

    ##AHGLAB plot Spectrum & Spectrum (after FIR_bandpassfilter_fdp)
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, AHGLAB_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], AHGLAB_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], AHGLAB_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], AHGLAB_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_AHGLAB f1 = {} (ref-in)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], AHGLAB_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], AHGLAB_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], AHGLAB_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], AHGLAB_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], AHGLAB_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], AHGLAB_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_AHGLAB f1 = %s (ref-in)_%s.png' %(f1[i],savename))

    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, FIR_AHGLAB_spectrum_l)
    plt.plot(f_axis[int(round(f1[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_AHGLAB f1 = {} after FIR_bandpassfilter_fdp (ref-in)'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], FIR_AHGLAB_spectrum_l[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], FIR_AHGLAB_spectrum_l[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], FIR_AHGLAB_spectrum_l[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], FIR_AHGLAB_spectrum_l[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE Spectrum_AHGLAB f1 = %s after FIR_bandpassfilter_fdp (ref-in)_%s.png' %(f1[i],savename))
    
    #plot Spectrum_Computer input L1 L2
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, spectrum_L1)
    plt.plot(f_axis[int(round(f1[i] / df))], spectrum_L1[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], spectrum_L1[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], spectrum_L1[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('Spectrum_Computer input L1 f1 = {}'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], spectrum_L1[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], spectrum_L1[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], spectrum_L1[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], spectrum_L1[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], spectrum_L1[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], spectrum_L1[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('Spectrum_Computer input L1 f1 = %s (Left ear)_%s.png' %(f1[i],savename))
    
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(f_axis, spectrum_L2)
    plt.plot(f_axis[int(round(f1[i] / df))], spectrum_L2[int(round(f1[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f2[i] / df))], spectrum_L2[int(round(f2[i] / df))], 'mo', markerfacecolor = 'None')
    plt.plot(f_axis[int(round(f3[i] / df))], spectrum_L2[int(round(f3[i] / df))], 'go', markerfacecolor = 'None')
    plt.title('Spectrum_Computer input L2 f1 = {}'.format(f1[i]), **title_font)
    plt.text(f_axis[int(round(f1[i] / df))], spectrum_L2[int(round(f1[i] / df))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], spectrum_L2[int(round(f1[i] / df))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f2[i] / df))], spectrum_L2[int(round(f2[i] / df))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], spectrum_L2[int(round(f2[i] / df))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(f_axis[int(round(f3[i] / df))], spectrum_L2[int(round(f3[i] / df))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], spectrum_L2[int(round(f3[i] / df))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('Spectrum_Computer input L2 f1 = %s (Left ear)_%s.png' %(f1[i],savename)) 
    
    #PSDv2 denoise Cathy after recursive averaging(PSDv2), plot spectrum
    RA_axis, RA_nfft, RA_PSD_power, PSD_Cathy_time, num_PSDgain_negative = PSDv2(0, FIR_Cathy_signal_l.real, FIR_AHGLAB_signal_l.real, FIR_signal_L1, FIR_signal_L2, fs, i)
    totla_num_PSDgain_negative.append(num_PSDgain_negative)
    print("num_PSDgain_negative=",num_PSDgain_negative)
    RA_PSD_spectrum_l = 10 * np.log10(np.abs(RA_PSD_power))[:int(RA_nfft / 2)] + 94
    
    #signal to noise ratio(SNR), the larger the better.
    signal_PSD_l3_l, abar_PSD, std_noi_PSD, noise2sd_PSD, SNR_PSD=SNR(RA_PSD_power, RA_PSD_spectrum_l, f3[i], fs, RA_nfft)
    print("\nsignal_PSD_l3_l=",signal_PSD_l3_l)
    print("abar_PSD=",abar_PSD)
    print("std_noi_PSD=",std_noi_PSD) 
    print("noise2sd_PSD=",noise2sd_PSD) 
    print("after denoise SNR_PSD=",SNR_PSD)
    optimized_total_SNR_PSD.append(SNR_PSD)
    optimized_total_noise2sd_PSD.append(noise2sd_PSD)
    
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(RA_axis, RA_PSD_spectrum_l)
    plt.plot(RA_axis[int(round(f1[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))], 'mo', markerfacecolor = 'None')
    plt.plot(RA_axis[int(round(f2[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))], 'mo', markerfacecolor = 'None')
    plt.plot(RA_axis[int(round(f3[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_f1 = {} after FIR_after PSD_stage 1 (Left ear)'.format(f1[i]), **title_font)
    plt.text(RA_axis[int(round(f1[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], RA_PSD_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(RA_axis[int(round(f2[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], RA_PSD_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(RA_axis[int(round(f3[i] / (fs/RA_nfft)))], RA_PSD_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], RA_PSD_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE RA Spectrum_f1 = %s after FIR_after PSD_stage 1 (Left ear)_%s.png' %(f1[i],savename))
    
    #before denoise SNR
    RA_axis, RA_nfft, RA_Cathy_signal_l = Step123(0, Cathy_signal_l.real, fs, i)
    RA_Cathy_power=RA_Cathy_signal_l[:int(RA_nfft / 2)]
    RA_Cathy_signal_l_spectrum_l = 10 * np.log10(np.abs(RA_Cathy_signal_l))[:int(RA_nfft / 2)] + 94
    signal_Cathy_l3_l, abar_Cathy, std_noi_Cathy, noise2sd_Cathy, SNR_Cathy=SNR(RA_Cathy_power, RA_Cathy_signal_l_spectrum_l, f3[i], fs, RA_nfft)
    print("\nsignal_Cathy_l3_l=",signal_Cathy_l3_l)
    print("abar_Cathy=",abar_Cathy)
    print("std_noi_Cathy=",std_noi_Cathy) 
    print("noise2sd_Cathy=",noise2sd_Cathy) 
    print("Before denoise SNR_Cathy=",SNR_Cathy)
    beforedenoise_total_SNR_Cathy.append(SNR_Cathy)
    beforedenoise_total_noise2sd_Cathy.append(noise2sd_Cathy)
    #Cathy after recursive averaging(Step123), plot spectrum
    plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
    plt.plot(RA_axis, RA_Cathy_signal_l_spectrum_l)
    plt.plot(RA_axis[int(round(f1[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))], 'mo', markerfacecolor = 'None')
    plt.plot(RA_axis[int(round(f2[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))], 'mo', markerfacecolor = 'None')
    plt.plot(RA_axis[int(round(f3[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))], 'go', markerfacecolor = 'None')
    plt.title('DPOAE Spectrum_Cathy f1 = {} (Left ear)'.format(f1[i]), **title_font)
    plt.text(RA_axis[int(round(f1[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))], "(f1,L1)=\n(%dHz,%.2fdB)" %(f1[i], RA_Cathy_signal_l_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='right', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(RA_axis[int(round(f2[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))], "(f2,L2)=\n(%dHz,%.2fdB)" %(f2[i], RA_Cathy_signal_l_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='left', fontsize=axis_font['size'], rotation=0, color='m')
    plt.text(RA_axis[int(round(f3[i] / (fs/RA_nfft)))], RA_Cathy_signal_l_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))], "(fdp,Ldp)=\n(%dHz,%.2fdB)" %(f3[i], RA_Cathy_signal_l_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))]), verticalalignment='bottom', horizontalalignment='center', fontsize=axis_font['size'], rotation=0, color='g')
    plt.xticks( **tick_font)
    plt.yticks( **tick_font)
    plt.xlabel('Frequency (Hz)', **axis_font)
    plt.ylabel('SPL (dB)', **axis_font)
    plt.xlim(0, 8000) # 0~8000 Hz
    plt.ylim(-30, 75) # -30~75 dB
    plt.show()
    plt.savefig('DPOAE RA Spectrum_Cathy f1 = %s (Left ear)_%s.png' %(f1[i],savename))                                                                                                                       

    #L1 L2 L3 collection
    L1.append(spectrum_L1[int(round(f1[i] / df))])
    L2.append(spectrum_L2[int(round(f2[i] / df))])                                                   
    origin_l1_l.append(Cathy_spectrum_l[int(round(f1[i] / df))])
    origin_l2_l.append(Cathy_spectrum_l[int(round(f2[i] / df))])
    origin_l3_l.append(Cathy_spectrum_l[int(round(f3[i] / df))])
    RA_origin_l1_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))])
    RA_origin_l2_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))])
    RA_origin_l3_l.append(RA_Cathy_signal_l_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))])
    RA_PSD_Cathy_l1_l.append(RA_PSD_spectrum_l[int(round(f1[i] / (fs/RA_nfft)))])
    RA_PSD_Cathy_l2_l.append(RA_PSD_spectrum_l[int(round(f2[i] / (fs/RA_nfft)))])
    RA_PSD_Cathy_l3_l.append(RA_PSD_spectrum_l[int(round(f3[i] / (fs/RA_nfft)))])
                                                                         
    #save after denoised wav file_PSD
    #sf.write('denoise PSD_quiet_f1=%sHz.wav' %(f1[i]), PSD_Cathy_time.real, fs)
    #sf.write('denoise PSD_70dBA_f1=%sHz.wav' %(f1[i]), PSD_Cathy_time.real, fs)
    sf.write('denoise PSD_75dBA_f1=%sHz.wav' %(f1[i]), PSD_Cathy_time.real, fs)     

for pp in range(len(beforedenoise_total_SNR_Cathy)):
    SNR_improvement_PSD.append(optimized_total_SNR_PSD[pp]-beforedenoise_total_SNR_Cathy[pp])


--------------  f1=600  --------------
len(Cathy_spectrum_l)= 239990

PSDv2 frameNumber= 623
len(G)= 3072
num_PSDgain_negative= 1185

signal_PSD_l3_l= 19.06732660318262
abar_PSD= 1.8225481521494883e-08
std_noi_PSD= 1.186777047989662e-08
noise2sd_PSD= 16.606790113036254
after denoise SNR_PSD= 2.4605364901463673

signal_Cathy_l3_l= 23.347013220873947
abar_Cathy= 4.3060881671377234e-05
std_noi_Cathy= 5.337473517258798e-05
noise2sd_Cathy= 50.340829177562554
Before denoise SNR_Cathy= -26.993815956688607

--------------  f1=800  --------------
len(Cathy_spectrum_l)= 239990

PSDv2 frameNumber= 623
len(G)= 3072
num_PSDgain_negative= 832

signal_PSD_l3_l= -15.34499358350591
abar_PSD= 5.14586972410349e-09
std_noi_PSD= 3.201602170716853e-09
noise2sd_PSD= 11.114587871562634
after denoise SNR_PSD= -26.459581455068545

signal_Cathy_l3_l= 16.38573036564678
abar_Cathy= 2.3905832190869744e-07
std_noi_Cathy= 2.0147029486954843e-07
noise2sd_Cathy= 27.785038666100093
Before denoise SNR_Cathy= -11.3993083

<ipython-input-9-7b536a968678>:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
<ipython-input-9-7b536a968678>:73: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
<ipython-input-9-7b536a968678>:92: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much me


PSDv2 frameNumber= 623
len(G)= 3072
num_PSDgain_negative= 2339

signal_PSD_l3_l= 2.9015630275981863
abar_PSD= 1.3438272620163717e-11
std_noi_PSD= 1.3773593716044853e-12
noise2sd_PSD= -14.71656552691752
after denoise SNR_PSD= 17.618128554515707


<ipython-input-9-7b536a968678>:218: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image



signal_Cathy_l3_l= 16.052123811600936
abar_Cathy= 5.947972997228414e-08
std_noi_Cathy= 2.7590839149838507e-08
noise2sd_Cathy= 21.74368988232345
Before denoise SNR_Cathy= -5.691566070722516


...

<ipython-input-9-7b536a968678>:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
<ipython-input-9-7b536a968678>:73: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image
<ipython-input-9-7b536a968678>:92: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much me


PSDv2 frameNumber= 123
len(G)= 3072
num_PSDgain_negative= 824

signal_PSD_l3_l= 11.599300507620455
abar_PSD= 3.7007901284032457e-09
std_noi_PSD= 6.055701075122116e-10
noise2sd_PSD= 9.682944569781782
after denoise SNR_PSD= 1.9163559378386736


<ipython-input-9-7b536a968678>:218: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image



signal_Cathy_l3_l= 12.132602550458472
abar_Cathy= 1.6161222710535543e-08
std_noi_Cathy= 3.0311184543786736e-09
noise2sd_Cathy= 16.084742151236213
Before denoise SNR_Cathy= -3.952139600777741


<ipython-input-9-7b536a968678>:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1800/my_dpi, 900/my_dpi), dpi=my_dpi) #1800x900 pixel image


In [10]:
print("totla_num_PSDgain_negative=",totla_num_PSDgain_negative)

#print("\nComputer input L1 = ", L1)
#print("Computer input L2 = ", L2) 

#print("\norigin_l1_l = ", origin_l1_l)
#print("origin_l2_l = ", origin_l2_l)
print("origin_l3_l = ", origin_l3_l)
print("\nRA_origin_l1_l = ", RA_origin_l1_l)
print("RA_origin_l2_l = ", RA_origin_l2_l)
print("RA_origin_l3_l = ", RA_origin_l3_l)

print("\nRA_PSD_Cathy_l1_l = ", RA_PSD_Cathy_l1_l)
print("RA_PSD_Cathy_l2_l = ", RA_PSD_Cathy_l2_l)
print("RA_PSD_Cathy_l3_l = ", RA_PSD_Cathy_l3_l)

print("optimized_total_SNR_PSD =",optimized_total_SNR_PSD)
print("optimized_total_noise2sd_PSD =",optimized_total_noise2sd_PSD)

print("\nbeforedenoise_total_SNR_Cathy =",beforedenoise_total_SNR_Cathy)
print("beforedenoise_total_noise2sd_Cathy =",beforedenoise_total_noise2sd_Cathy)
print("SNR_improvement_PSD =",SNR_improvement_PSD)

totla_num_PSDgain_negative= [1185, 832, 2339, 1338, 1288, 580, 1327, 446, 520, 824]
origin_l3_l =  [7.531829667102997, 9.625219156093692, 12.109235235726857, 8.682084200433806, -2.7302628234296265, 4.847100876692139, 7.562633736234119, 11.401369532461516, 4.473925279228752, 11.246725757034866]

RA_origin_l1_l =  [64.87934555892112, 65.70116396996063, 65.96245495384291, 68.00213944980615, 63.46546094323857, 64.45628963573333, 66.04415446165163, 63.45711798155672, 61.61334843574063, 64.24487871738131]
RA_origin_l2_l =  [55.232515976103315, 55.965503948539705, 56.90676542205528, 55.90679200442173, 53.97221050923748, 54.05798193608805, 52.886189534478895, 51.59974723844589, 53.51824894834335, 58.05643690494417]
RA_origin_l3_l =  [23.347013220873947, 16.38573036564678, 16.052123811600936, 16.049966023923346, 12.906520652435589, 21.412964581589463, 16.84740820646057, 23.81531079450653, 22.40628859003384, 12.132602550458472]

RA_PSD_Cathy_l1_l =  [25.331111350587605, 16.900578094015387, 12.64

In [ ]:
##RA Comparison with 2speaker_L&R 34dBA quiet
#matplotlib qt
#labels = [0.732, 0.976, 1.098, 1.464, 3.05, 3.904, 4.392, 4.88, 5.856, 7.32] #f2
labels = [0.468, 0.624, 0.702, 0.936, 1.950, 2.496, 2.808, 3.12, 3.744, 4.68] #fdp(2f1-f2)
plt.figure(figsize=(1200/my_dpi, 900/my_dpi), dpi=my_dpi) #1200x900 pixel image
g, = plt.plot(RA_origin_l3_l, 'ko') #20201205 DPOAE_quiet
g, = plt.plot(RA_origin_l3_l, 'k') #20201205 DPOAE_quiet
h, = plt.plot(RA_PSD_Cathy_l3_l, 'g--') #PSD
n, = plt.plot(optimized_total_noise2sd_PSD , 'g:')
plt.legend(handles = [g, h, n], labels = ['quiet', 'PSD', 'noise_PSD'], prop = {'size' : tick_font['size']})
plt.title('DPOAE_%s' %name.split('_')[0], **title_font)
plt.xticks(np.arange(len(f1)), labels,  **tick_font)
plt.yticks( **tick_font)
#plt.xlabel('f2 frequency (kHz)', **axis_font)
plt.xlabel('fdp(2f1-f2) frequency (kHz)', **axis_font)
plt.ylabel('SPL (dB)', **axis_font)
plt.show()
plt.savefig('Comparison_RA_DPOAE_PSD_34dBA quiet.png')

In [11]:
##RA Comparison with 2speaker_L&R 70dBA, 75dBA, 80dBA, 85dBA white noise
#matplotlib qt
#labels = [0.732, 0.976, 1.098, 1.464, 3.05, 3.904, 4.392, 4.88, 5.856, 7.32] #f2
labels = [0.468, 0.624, 0.702, 0.936, 1.950, 2.496, 2.808, 3.12, 3.744, 4.68] #fdp(2f1-f2)
plt.figure(figsize=(1200/my_dpi, 900/my_dpi), dpi=my_dpi) #1200x900 pixel image
#plt.figure(figsize=(1800/my_dpi, 1000/my_dpi), dpi=my_dpi) #1800x1000 pixel image

r, = plt.plot(RA_origin_l3_l, 'r--')
t, = plt.plot([15.42907192038146, 15.528941793215878, 16.860277454068424, 9.835509604682457, 2.0966605871184925, 7.648677264691429, 10.88963672188359, 14.198263639816702, 12.646268840947826, 13.052827184310019], 'ko') #20201205 DPOAE_quiet
t, = plt.plot([15.42907192038146, 15.528941793215878, 16.860277454068424, 9.835509604682457, 2.0966605871184925, 7.648677264691429, 10.88963672188359, 14.198263639816702, 12.646268840947826, 13.052827184310019], 'k') #20201205 DPOAE_quiet
y, = plt.plot(RA_PSD_Cathy_l3_l, 'g--') #PSD
n, = plt.plot(optimized_total_noise2sd_PSD , 'g:')
plt.legend(handles = [r, t, y, n], labels = ['75dBA white noise','quiet', 'PSD', 'noise_PSD'], prop = {'size' : tick_font['size']})
plt.title('DPOAE_%s' %name.split('_')[0], **title_font)
plt.xticks(np.arange(len(f1)), labels,  **tick_font)
plt.yticks( **tick_font)
#plt.xlabel('f2 frequency (kHz)', **axis_font)
plt.xlabel('fdp(2f1-f2) frequency (kHz)', **axis_font)
plt.ylabel('SPL (dB)', **axis_font)
plt.show()
plt.savefig('Comparison_RA_DPOAE_PSD_75dBA white noise.png')

<ipython-input-11-fe7e4a48417b>:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(1200/my_dpi, 900/my_dpi), dpi=my_dpi) #1200x900 pixel image
